# 티어별 챔피언 픽 시각화
---
대화형 plot : ipywidgets을 이용해 plotly에 GUI를 생성

---
## 사용 라이브러리

In [2]:
pip install ipywidgets

  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/4a/0e/57ed498fafbc60419a9332d872e929879ceba2d73cb11d284d7112472b3e/ipywidgets-8.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.9 from https://files.pythonhosted.org/packages/29/03/107d96077c4befed191f7ad1a12c7b52a8f9d2778a5836d59f9855c105f6/widgetsnbextension-4.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.9 from https://files.pythonhosted.org/packages/e8/05/0ebab152288693b5ec7b339aab857362947031143b282853b4c2dd4b5b40/jupyterlab_widgets-3.0.9-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/139.4 kB ? eta -:--:--
   ---------------------------------------- 139.4/139.4 kB 8.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/214.9 kB ? eta -:--:--
   --------------------------------------- 214.9/214.9 kB 12.8 MB/s eta 0:00:00
   ----------------------------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install plotly

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/a8/07/72953cf70e3bd3a24cbc3e743e6f8539abe6e3e6d83c3c0c83426eaffd39/plotly-5.18.0-py3-none-any.whl.metadata
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.6 MB 11.1 MB/s eta 0:00:02
   -- ------------------------------------- 1.1/15.6 MB 11.7 MB/s eta 0:00:02
   ---- ----------------------------------- 1.6/15.6 MB 11.7 MB/s eta 0:00:02
   ----- ---------------------------------- 2.2/15.6 MB 11.7 MB/s eta 0:00:02
   ------- -------------------------------- 2.8/15.6 MB 11.8 MB/s eta 0:00:02
   -------- ------------------------------- 3.3/15.6 MB 11.8 MB/s eta


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import ipywidgets
ipywidgets.__version__

'8.1.1'

In [7]:
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.express as px
import pandas as pd
import numpy as np

## 사용 데이터 불러오기

In [8]:
count_iron = pd.read_csv("match_data/iron_count.csv", encoding = 'cp949')
count_bronze = pd.read_csv("match_data/bronze_count.csv", encoding = 'cp949')
count_silver = pd.read_csv("match_data/silver_count.csv", encoding = 'cp949')
count_gold = pd.read_csv("match_data/gold_count.csv", encoding = 'cp949')
count_platinum = pd.read_csv("match_data/platinum_count.csv", encoding = 'cp949')
count_platinum

,position,championID,count
0,BOTTOM,222,7573
1,BOTTOM,81,6377
2,BOTTOM,498,4075
3,BOTTOM,145,3734
4,BOTTOM,523,3441
...,...,...,...
709,UTILITY,106,1
710,UTILITY,121,1
711,UTILITY,221,1
712,UTILITY,360,1


## [플래티넘] 빈도 조회
---
조건 : 포지션, 챔피언

In [ ]:
position = widgets.Dropdown(
        options = list(count_platinum.position.unique()),
        value = 'TOP',
        description = 'POSITION:'
)
championID = widgets.Dropdown(
    description = 'ChampionID: ',
    value = 222,
    options = count_platinum['championID'].unique().tolist()
)

fig_1 = go.Figure()
fig_1.add_trace(
    go.Bar(x = count_platinum['championID'], y = count_platinum['count'])
)
fig_1.update_layout(title = {"text" :"[플래티넘] 티어별 챔피언별 카운트 조회"},
                       xaxis = {'title' : {'text' : 'championID'}},
                       yaxis = {'title' : {'text' : 'count'}})
g_1 = go.FigureWidget(data = fig_1,
                   layout = fig_1.layout)
def validate():
    if position.value in count_platinum.position.unique() and championID.value in count_platinum['championID'].unique():
        return True
    else:
        return False
    
def response(change):
    if validate():
        filter_list = [i and j for i, j in zip(count_platinum['position'] == position.value, 
                                           count_platinum['championID'] == championID.value)]
        temp_df = count_platinum[filter_list]
    x1 = temp_df['count']
    with g_1.batch_update():
        g_1.data[0].y = x1
        g_1.layout.xaxis.title = 'champion'
        g_1.layout.yaxis.title = 'count'
position.observe(response, names = 'value')
championID.observe(response, names = 'value')
container = widgets.HBox([position, championID])
widgets.VBox([container, g_1])

## [플래티넘] 포지션별 챔피언 픽 분포
---
조건 : 포지션

In [34]:
position = widgets.Dropdown(
        options = list(count_platinum.position.unique()),
        value = 'TOP',
        description = 'POSITION:'
)
fig = go.Figure()
fig.add_trace(
    go.Bar(x=count_platinum['championID'], y = count_platinum['count'])
)
fig.update_layout(title = {"text" :"[플래티넘] 티어별 챔피언 픽 분포"},
                       xaxis = {'title' : {'text' : 'championID'}},
                       yaxis = {'title' : {'text' : 'count'}})
g = go.FigureWidget(data = fig,
                   layout = fig.layout)
def validate():
    if position.value in count_platinum.position.unique():
        return True
    else:
        return False
    
def response(change):
    if validate():
        temp_df = count_platinum.loc[count_platinum.position == position.value]
    x1 = temp_df['count']
    with g.batch_update():
        g.data[0].y = x1
        g.layout.title.text = "[플래티넘] 티어별 챔피언 픽 분포"
        g.layout.xaxis.title = 'championID'
        g.layout.yaxis.title = 'count'
position.observe(response, names = 'value')
championID.observe(response, names = 'value')
container = widgets.HBox([position])
widgets.VBox([container, g])

## 티어별 챔피언 픽 분포
---
조건 : 티어, 포지션

In [15]:
# 티어 추가
count_iron['tier'] = "IRON"
count_bronze['tier'] = "BRONZE"
count_silver['tier'] = "SILVER"
count_gold['tier'] = "GOLD"
count_platinum['tier'] = "PLATINUM"
count_platinum

,position,championID,count,tier
0,BOTTOM,222,7573,PLATINUM
1,BOTTOM,81,6377,PLATINUM
2,BOTTOM,498,4075,PLATINUM
3,BOTTOM,145,3734,PLATINUM
4,BOTTOM,523,3441,PLATINUM
...,...,...,...,...
709,UTILITY,106,1,PLATINUM
710,UTILITY,121,1,PLATINUM
711,UTILITY,221,1,PLATINUM
712,UTILITY,360,1,PLATINUM


In [16]:
# 데이터 병합
count_data = pd.concat([count_iron, count_bronze, count_silver, count_gold, count_platinum])
count_data

,position,championID,count,tier
0,BOTTOM,81,6320,IRON
1,BOTTOM,222,5777,IRON
2,BOTTOM,21,4098,IRON
3,BOTTOM,51,3924,IRON
4,BOTTOM,145,3552,IRON
...,...,...,...,...
709,UTILITY,106,1,PLATINUM
710,UTILITY,121,1,PLATINUM
711,UTILITY,221,1,PLATINUM
712,UTILITY,360,1,PLATINUM


In [17]:
count_data.to_csv("match_data/count_data.csv", encoding = 'cp949')

In [18]:
count_data = pd.read_csv("match_data/count_data.csv", encoding = 'cp949')

In [35]:
position = widgets.Dropdown(
        options = list(count_data.position.unique()),
        value = 'TOP',
        description = 'POSITION :'
)
tier = widgets.TagsInput(
    value = ['IRON'],
    allowed_tags = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM'],
    allowd_duplicates = False,
    description = 'TIER : '
)
fig_2 = go.Figure()
fig_2.add_trace(
    go.Bar(x=count_data['championID'], y = count_data['count'], opacity = 0.75)
)
fig_2.update_layout(title = {"text" :"티어별 챔피언 픽 분포"},
                       xaxis = {'title' : {'text' : 'championID'}},
                       yaxis = {'title' : {'text' : 'count'}},
                       barmode = 'overlay'
                 )
g_2 = go.FigureWidget(data = fig_2,
                   layout = fig_2.layout)
def validate():
    if position.value in count_data.position.unique():
        return True
    else:
        return False
    
def response(change):
    if validate():
        if len(tier.value) == 1:
            filter_list1 = [i and j for i, j in zip(count_data['tier'] == tier.value[0], 
                                           count_data['position'] == position.value)]
            temp_df1 = count_data[filter_list1]
            x1 = temp_df1['count']
        elif len(tier.value) == 2:
            filter_list1 = [i and j for i, j in zip(count_data['tier'] == tier.value[0], 
                                           count_data['position'] == position.value)]
            filter_list2 = [i and j for i, j in zip(count_data['tier'] == tier.value[1], 
                                           count_data['position'] == position.value)]
            temp_df1 = count_data[filter_list1]
            temp_df2 = count_data[filter_list2]
            x1 = temp_df1['count']
            x2 = temp_df2['count']
            g_2.add_bar(x = temp_df2['championID'], y = x2, opacity = 0.75)
        elif len(tier.value) == 3:
            filter_list1 = [i and j for i, j in zip(count_data['tier'] == tier.value[0], 
                                           count_data['position'] == position.value)]
            filter_list2 = [i and j for i, j in zip(count_data['tier'] == tier.value[1], 
                                           count_data['position'] == position.value)]
            filter_list3 = [i and j for i, j in zip(count_data['tier'] == tier.value[2], 
                                           count_data['position'] == position.value)]
            temp_df1 = count_data[filter_list1]
            temp_df2 = count_data[filter_list2]
            temp_df3 = count_data[filter_list3]
            x1 = temp_df1['count']
            x2 = temp_df2['count']
            x3 = temp_df3['count']
            g_2.add_bar(x = temp_df2['championID'], y = x2, opacity = 0.75)
            g_2.add_bar(x = temp_df3['championID'], y = x3, opacity = 0.75)
        else:
            filter_list1 = [i and j for i, j in zip(count_data['tier'] == tier.value[0], 
                                           count_data['position'] == position.value)]
            filter_list2 = [i and j for i, j in zip(count_data['tier'] == tier.value[1], 
                                           count_data['position'] == position.value)]
            filter_list3 = [i and j for i, j in zip(count_data['tier'] == tier.value[2], 
                                           count_data['position'] == position.value)]
            filter_list4 = [i and j for i, j in zip(count_data['tier'] == tier.value[3], 
                                           count_data['position'] == position.value)]
            temp_df1 = count_data[filter_list1]
            temp_df2 = count_data[filter_list2]
            temp_df3 = count_data[filter_list3]
            temp_df4 = count_data[filter_list4]
            x1 = temp_df1['count']
            x2 = temp_df2['count']
            x3 = temp_df3['count']
            x4 = temp_df4['count']
            g_2.add_bar(x = temp_df2['championID'], y = x2, opacity = 0.75)
            g_2.add_bar(x = temp_df3['championID'], y = x3, opacity = 0.75)
            g_2.add_bar(x = temp_df4['championID'], y = x4, opacity = 0.75)
    with g_2.batch_update():
        if len(tier.value) == 1:
            g_2.data[0].y = x1
        elif len(tier.value) == 2:
            g_2.data[0].y = x1
            g_2.data[1].y = x2
        elif len(tier.value) == 3:
            g_2.data[0].y = x1
            g_2.data[1].y = x2
            g_2.data[2].y = x3
            g_2.data[3].y = x4
        g_2.layout.title.text = "티어별 챔피언 픽 분포"
        g_2.layout.xaxis.title = 'championID'
        g_2.layout.yaxis.title = 'count'
        g_2.layout.barmode = 'overlay'

position.observe(response, names = 'value')
championID.observe(response, names = 'value')
container2 = widgets.HBox([position])
container = widgets.HBox([tier])
widgets.VBox([container,container2, g_2])